## STEP 1

In [ ]:
# Importo i moduli necessari
import os, shutil, csv
from pathlib import Path as path

# Creo una lista alfabeticamente ordinata dei file iterando su ogni elemento della directory 'files'.
files = sorted(os.listdir('files'))

# Assegno alle variabili i relativi path
p = path('.') / 'files'
docs = (p / 'docs')
images = (p / 'images')
audio = (p / 'audio')

# Creo il file recap.csv o se esiste già lo aggiorno, inzializzo l'oggetto writer e scrivo l header solo se non eisiste gia.
with open (p / 'recap.csv', 'a', newline = '') as csvFile:
    writer = csv.writer(csvFile, delimiter = ',')
    if os.stat(p / 'recap.csv').st_size == 0:
        writer.writerow(['Name','Type','Size(Byte)'])

# Itero su ogni elemento della lista ordinata dei ed estraggo ne estraggo il path su cui effettuare le varie operazioni.
    for file in files:
        file = path(file)
        pathFile = (p / file)

    # Distinguo le varie estensioni dei file e li sposto di conseguenza nelle sub-directory
    # di apaprtenenza (se non esistono gia le creo) salvo le informazioni richieste in 
    # una riga del file csv e poi le stampo a video.
        if file.suffix in ('.txt','.odt'):
            os.makedirs(docs, exist_ok=True)
            shutil.move(pathFile, docs)
            writer.writerow([file.stem, 'doc', os.path.getsize(p/'docs'/file)])
            print(file.stem, 'doc', os.path.getsize(p /'docs'/ file) , ' (Byte)')

        elif file.suffix in ('.jpg','.png','.jpeg'):
            os.makedirs(images, exist_ok=True)
            shutil.move(pathFile, images)
            writer.writerow([file.stem, 'image', os.path.getsize(p/'images'/file)])
            print(file.stem, 'image', os.path.getsize(p/'images'/file) , ' (Byte)')

        elif file.suffix == '.mp3':
            os.makedirs(audio, exist_ok=True)
            shutil.move(pathFile, audio)
            writer.writerow([file.stem, 'audio', os.path.getsize(p / 'audio' /file)])
            print(file.stem, 'audio', os.path.getsize(p /'audio'/ file) , ' (Byte)')

## Step 3

In [ ]:
# Importo i mouduli necessari
from PIL import Image
import numpy as np, os
from pathlib import Path as path
from tabulate import tabulate

p = (path('.') / 'files' / 'images')

# Inzializzo le liste che serviranno per stamapre la tabella in modo corretto.
headers = ['name', 'height', 'width', 'greyscale', 'R', 'G', 'B', 'Alpha']
table = []

# Itero su ogni immagine all'interno della cartella 'images' e le trasformo in array.
for image in sorted(os.listdir(p)):
    pathImage = p / image
    img = np.array(Image.open(pathImage))
   
    # Identifico i dati d'interesse da riporatre nella tabella e li assegno alle relative variabili.
    # img_dim mi servirà per distinguere tra immagini RGB e in bianco e nero.
    name = image.split('.')[0]
    height, width = img.shape[:2]
    img_dim = img.ndim
    
    # Identifico il caso di immagine in scala di grigi (array bidimensionle per questo img_dim == 2). Uso la funzione mean() di NumPy per ottenere
    # la media di tutti i valori che compogono l'array.
    if img_dim == 2:
        img_mean = np.mean(img)
        greyscale = np.around(img_mean, 2)
        table_bw = [name, height, width, greyscale]
        table.append(table_bw)
    
    # Distinguo i casi di immagini RGB e RGBAlpha basadomi sul numero delle colonne che compongono gli array.
    # specificando l'argomento axis=(0,1), mean() di NumPy restitusice una media considerando ogni elemento per ogni colonna che compone l'array. 
    # Avendo RGBA 4 canali avrà 4 colonne mentre RGB solo 3.
    else:
        img_mean = np.mean(img, axis = (0,1))
        
        if len(img_mean) == 3:
            table_RGB = []
            R = np.around(img_mean[0],2)
            G = np.around(img_mean[1],2)
            B = np.around(img_mean[2],2)
            table_RGB = [name, height, width, None, R, G, B]
            table.append(table_RGB)
        else:
            R = np.around(img_mean[0],2)
            G = np.around(img_mean[1],2)
            B = np.around(img_mean[2],2)
            A = np.around(img_mean[3],2)
            table_RGBA = [name, height, width, None, R, G, B, A]
            table.append(table_RGBA)

# Stampo con l'uso di tabulate i dati richiesti e inserisco negli spazi vuoti il valore 0.00.
print(tabulate(table, headers, tablefmt = "fancy_grid", missingval='0.00'))

**Dovrai consegnare**:
- un notebook con gli Step 1 e 3; per semplicità puoi chiamarlo come questo
- addfile.py con quanto richiesto dallo Step 2.